## Machine Learning - XGboost

#### Importación de librerías

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
import xgboost as xgb
from xgboost import plot_importance
from matplotlib import pyplot
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV as GridSearch
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, precision_recall_curve,roc_curve


/home/grobles/FIUBA/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


#### Importación de features

In [2]:
features = pd.read_csv('../features/features_todos.csv')
features.head()

,person,marca_favorita_apple,marca_favorita_asus,marca_favorita_lenovo,marca_favorita_lg,marca_favorita_motorola,marca_favorita_no_tiene,marca_favorita_quantum,marca_favorita_samsung,marca_favorita_sony,...,generic listing,searched products,viewed product,visited site,completa_flujo,cantidad_lead_y,favorite_device_Computer,favorite_device_Smartphone,favorite_device_Tablet,favorite_device_Unknown
0,0008ed71,1,0,0,0,0,0,0,0,0,...,1,0,0,2,0,0,1,0,0,0
1,91926,1,0,0,0,0,0,0,0,0,...,0,0,372,34,0,0,1,0,0,0
2,00091a7a,1,0,0,0,0,0,0,0,0,...,0,0,3,1,0,0,0,1,0,0
3,000ba417,0,0,0,0,0,0,0,1,0,...,14,0,153,6,0,0,1,0,0,0
4,000c79fe,1,0,0,0,0,0,0,0,0,...,1,9,3,1,0,0,0,1,0,0


#### Categorización de features categóricos

In [3]:
# Según Marca
features['marca_favorita_apple'] = features['marca_favorita_apple'].astype('category')
features['marca_favorita_asus'] = features['marca_favorita_asus'].astype('category')
features['marca_favorita_lenovo'] = features['marca_favorita_lenovo'].astype('category')
features['marca_favorita_lg'] = features['marca_favorita_lg'].astype('category')
features['marca_favorita_motorola'] = features['marca_favorita_motorola'].astype('category')
features['marca_favorita_no_tiene'] = features['marca_favorita_no_tiene'].astype('category')
features['marca_favorita_quantum'] = features['marca_favorita_quantum'].astype('category')
features['marca_favorita_samsung'] = features['marca_favorita_samsung'].astype('category')
features['marca_favorita_sony'] = features['marca_favorita_sony'].astype('category')

features['cant_modelo_iphone'] = features['cant_modelo_iphone'].astype('category')
features['cant_modelo_samsung'] = features['cant_modelo_samsung'].astype('category')
features['cant_modelo_motorola'] = features['cant_modelo_motorola'].astype('category')
features['cant_modelo_otro'] = features['cant_modelo_otro'].astype('category')
features['modelo_moda'] = features['modelo_moda'].astype('category')

features['porcentaje_modelo_iphone'] = features['porcentaje_modelo_iphone'].astype('category')
features['porcentaje_modelo_samsung'] = features['porcentaje_modelo_samsung'].astype('category')
features['porcentaje_modelo_motorola'] = features['porcentaje_modelo_motorola'].astype('category')
features['porcentaje_modelo_otro'] = features['porcentaje_modelo_otro'].astype('category')

# Nivel de Actividad
features['nivel_de_actividad_muy_bajo'] = features['nivel_de_actividad_muy_bajo'].astype('category')
features['nivel_de_actividad_bajo'] = features['nivel_de_actividad_bajo'].astype('category')
features['nivel_de_actividad_medio'] = features['nivel_de_actividad_medio'].astype('category')
features['nivel_de_actividad_alto'] = features['nivel_de_actividad_alto'].astype('category')
features['nivel_de_actividad_muy_alto'] = features['nivel_de_actividad_muy_alto'].astype('category')
# No es Categorico: features['dias_en_plataforma'] = features['dias_en_plataforma'].astype('category')
# No es Categorico: features['dias_distintos_con_eventos'] = features['dias_distintos_con_eventos'].astype('category')
# No es Categorico: features['eventos_totales_generados'] = features['eventos_totales_generados'].astype('category')
# No es Categorico: features['first_and_last_event_distance_in_days'] = features['first_and_last_event_distance_in_days'].astype('category')
# No es Categorico: features['mean_events_per_day'] = features['mean_events_per_day'].astype('category')

# Accesos Según Región
# No es Categorico: features['cant_accesos_southeast'] = features['cant_accesos_southeast'].astype('category')
# No es Categorico: features['cant_accesos_northeast'] = features['cant_accesos_northeast'].astype('category')
# No es Categorico: features['cant_accesos_south'] = features['cant_accesos_south'].astype('category')
# No es Categorico: features['cant_accesos_center_west'] = features['cant_accesos_center_west'].astype('category')
# No es Categorico: features['cant_accesos_north'] = features['cant_accesos_north'].astype('category')

# Eventos Según Franja Horaria
# No es Categorico: features['cant_ev_madrugada'] = features['cant_ev_madrugada'].astype('category')
# No es Categorico: features['cant_ev_manana'] = features['cant_ev_manana'].astype('category')
# No es Categorico: features['cant_ev_tarde'] = features['cant_ev_tarde'].astype('category')
# No es Categorico: features['cant_ev_noche'] = features['cant_ev_noche'].astype('category')
features['franja_horaria_moda'] = features['franja_horaria_moda'].astype('category')

# Según Estado del Celular
# No es Categorico: features['cant_buenos'] = features['cant_buenos'].astype('category')
# No es Categorico: features['cant_excelentes'] = features['cant_excelentes'].astype('category')
# No es Categorico: features['cant_muy_buenos'] = features['cant_muy_buenos'].astype('category')
# No es Categorico: features['cant_nuevos'] = features['cant_nuevos'].astype('category')
features['condicion_de_producto_moda'] = features['condicion_de_producto_moda'].astype('category')

# Según Semana del Mes
# No es Categorico: features['cant_ev_primer_semana_mes'] = features['cant_ev_primer_semana_mes'].astype('category')
# No es Categorico: features['cant_ev_segunda_semana_mes'] = features['cant_ev_segunda_semana_mes'].astype('category')
# No es Categorico: features['cant_ev_tercera_semana_mes'] = features['cant_ev_tercera_semana_mes'].astype('category')
# No es Categorico: features['cant_ev_cuarta_semana_mes'] = features['cant_ev_cuarta_semana_mes'].astype('category')
features['semana_del_mes_moda'] = features['semana_del_mes_moda'].astype('category')
# No es Categorico: features['porcentaje_eventos_en_fin_de_semana'] = features['porcentaje_eventos_en_fin_de_semana'].astype('category')

# Interes En Los Últimos Días
# No es Categorico: features['interest_last_2_days'] = features['interest_last_2_days'].astype('category')
# No es Categorico: features['interest_last_4_days'] = features['interest_last_4_days'].astype('category')
# No es Categorico: features['interest_last_6_days'] = features['interest_last_6_days'].astype('category')
# No es Categorico: features['interest_last_8_days'] = features['interest_last_8_days'].astype('category')
# No es Categorico: features['interest_last_10_days'] = features['interest_last_10_days'].astype('category')
# No es Categorico: features['interest_last_12_days'] = features['interest_last_12_days'].astype('category')
# No es Categorico: features['interest_last_14_days'] = features['interest_last_14_days'].astype('category')
# No es Categorico: features['interest_last_16_days'] = features['interest_last_16_days'].astype('category')
# No es Categorico: features['interest_last_18_days'] = features['interest_last_18_days'].astype('category')
# No es Categorico: features['interest_last_20_days'] = features['interest_last_20_days'].astype('category')
# No es Categorico: features['interest_last_22_days'] = features['interest_last_22_days'].astype('category')
# No es Categorico: features['interest_last_24_days'] = features['interest_last_24_days'].astype('category')
# No es Categorico: features['interest_last_26_days'] = features['interest_last_26_days'].astype('category')
# No es Categorico: features['interest_last_28_days'] = features['interest_last_28_days'].astype('category')
# No es Categorico: features['interest_last_30_days'] = features['interest_last_30_days'].astype('category')



In [4]:
features.head()

,person,marca_favorita_apple,marca_favorita_asus,marca_favorita_lenovo,marca_favorita_lg,marca_favorita_motorola,marca_favorita_no_tiene,marca_favorita_quantum,marca_favorita_samsung,marca_favorita_sony,...,generic listing,searched products,viewed product,visited site,completa_flujo,cantidad_lead_y,favorite_device_Computer,favorite_device_Smartphone,favorite_device_Tablet,favorite_device_Unknown
0,0008ed71,1,0,0,0,0,0,0,0,0,...,1,0,0,2,0,0,1,0,0,0
1,91926,1,0,0,0,0,0,0,0,0,...,0,0,372,34,0,0,1,0,0,0
2,00091a7a,1,0,0,0,0,0,0,0,0,...,0,0,3,1,0,0,0,1,0,0
3,000ba417,0,0,0,0,0,0,0,1,0,...,14,0,153,6,0,0,1,0,0,0
4,000c79fe,1,0,0,0,0,0,0,0,0,...,1,9,3,1,0,0,0,1,0,0


In [5]:
features.shape

(38829, 88)

Obtengo el **nombre de las columnas** de los features

In [6]:
for column in features.columns:
    print ("'" + column + "'" + ",")


'person',
'marca_favorita_apple',
'marca_favorita_asus',
'marca_favorita_lenovo',
'marca_favorita_lg',
'marca_favorita_motorola',
'marca_favorita_no_tiene',
'marca_favorita_quantum',
'marca_favorita_samsung',
'marca_favorita_sony',
'nivel_de_actividad_muy_bajo',
'nivel_de_actividad_bajo',
'nivel_de_actividad_medio',
'nivel_de_actividad_alto',
'nivel_de_actividad_muy_alto',
'dias_en_plataforma',
'dias_distintos_con_eventos',
'eventos_totales_generados',
'cant_accesos_southeast',
'cant_accesos_northeast',
'cant_accesos_south',
'cant_accesos_center_west',
'cant_accesos_north',
'cant_total_eventos',
'first_and_last_event_distance_in_days',
'mean_events_per_day',
'cant_ev_madrugada',
'cant_ev_manana',
'cant_ev_tarde',
'cant_ev_noche',
'franja_horaria_moda',
'cant_buenos',
'cant_excelentes',
'cant_muy_buenos',
'cant_nuevos',
'condicion_de_producto_moda',
'cant_ev_primer_semana_mes',
'cant_ev_segunda_semana_mes',
'cant_ev_tercera_semana_mes',
'cant_ev_cuarta_semana_mes',
'semana_del_mes_moda'

In [7]:
features_names = [
    'person',
    'marca_favorita_apple',
    'marca_favorita_asus',
    'marca_favorita_lenovo',
    'marca_favorita_lg',
    'marca_favorita_motorola',
    'marca_favorita_no_tiene',
    'marca_favorita_quantum',
    'marca_favorita_samsung',
    'marca_favorita_sony',
    'nivel_de_actividad_muy_bajo',
    'nivel_de_actividad_bajo',
    'nivel_de_actividad_medio',
    'nivel_de_actividad_alto',
    'nivel_de_actividad_muy_alto',
    'dias_en_plataforma',
    'dias_distintos_con_eventos',
    'eventos_totales_generados',
    'cant_accesos_southeast',
    'cant_accesos_northeast',
    'cant_accesos_south',
    'cant_accesos_center_west',
    'cant_accesos_north',
    'cant_total_eventos',
    'first_and_last_event_distance_in_days',
    'mean_events_per_day',
    'cant_ev_madrugada',
    'cant_ev_manana',
    'cant_ev_tarde',
    'cant_ev_noche',
    'franja_horaria_moda',
    'cant_buenos',
    'cant_excelentes',
    'cant_muy_buenos',
    'cant_nuevos',
    'condicion_de_producto_moda',
    'cant_ev_primer_semana_mes',
    'cant_ev_segunda_semana_mes',
    'cant_ev_tercera_semana_mes',
    'cant_ev_cuarta_semana_mes',
    'semana_del_mes_moda',
    'porcentaje_eventos_en_fin_de_semana',
    'cant_modelo_iphone',
    'cant_modelo_samsung',
    'cant_modelo_motorola',
    'cant_modelo_otro',
    'porcentaje_modelo_iphone',
    'porcentaje_modelo_samsung',
    'porcentaje_modelo_motorola',
    'porcentaje_modelo_otro',
    'modelo_moda',
    'interest_last_2_days',
    'interest_last_4_days',
    'interest_last_6_days',
    'interest_last_8_days',
    'interest_last_10_days',
    'interest_last_12_days',
    'interest_last_14_days',
    'interest_last_16_days',
    'interest_last_18_days',
    'interest_last_20_days',
    'interest_last_22_days',
    'interest_last_24_days',
    'interest_last_26_days',
    'interest_last_28_days',
    'interest_last_30_days',
    'has_converted_before',
    'most_viewed_sku',
    'most_viewed_color_Preto',
    'most_viewed_color_Branco',
    'most_viewed_color_Dourado',
    'cantidad_checkouts',
    'cantidad_conversions',
    'tasa_cc',
    'cantidad_lead_x',
    'month_moda',
    'brand listing',
    'checkout',
    'generic listing',
    'searched products',
    'viewed product',
    'visited site',
    'completa_flujo',
    'cantidad_lead_y',
    'favorite_device_Computer',
    'favorite_device_Smartphone',
    'favorite_device_Tablet',
    'favorite_device_Unknown'
]

### Creación del Training Set

In [8]:
labels_training_set = pd.read_csv('../dataset/labels_training_set.csv')
test_set = pd.read_csv('../dataset/trocafone_kaggle_test.csv')

En **labels_training_set** están los usuarios con los labels, los resultados que conocemos de compras (1) o no (0) en los primeros 15 días de Junio.

En **test_set** están solamente las personas que queremos predecir si compraron o no.

Realizamos un merge entre el modelo con los **features** y **labels_training_set** para generar un modelo completo con los labels, con el fin de entrenar los algoritmos. Este modelo se guarda en **training_set**.

In [9]:
training_set_global = pd.merge(features, labels_training_set, how='inner', on='person')
training_set_global.head()

,person,marca_favorita_apple,marca_favorita_asus,marca_favorita_lenovo,marca_favorita_lg,marca_favorita_motorola,marca_favorita_no_tiene,marca_favorita_quantum,marca_favorita_samsung,marca_favorita_sony,...,searched products,viewed product,visited site,completa_flujo,cantidad_lead_y,favorite_device_Computer,favorite_device_Smartphone,favorite_device_Tablet,favorite_device_Unknown,label
0,0008ed71,1,0,0,0,0,0,0,0,0,...,0,0,2,0,0,1,0,0,0,0
1,000c79fe,1,0,0,0,0,0,0,0,0,...,9,3,1,0,0,0,1,0,0,0
2,0019e639,0,0,0,0,1,0,0,0,0,...,11,189,19,1,0,1,0,0,0,0
3,001b0bf9,0,0,0,0,0,0,0,1,0,...,0,2,0,0,0,0,0,0,0,0
4,001ca5ee,1,0,0,0,0,0,0,0,0,...,0,52,15,0,0,0,1,0,0,0


In [10]:
training_set_global.shape

(19257, 89)

Merge de **test_set**, que son las personas que queremos predecir si compraron o no con el resto del modelo con features.

In [19]:
test_set = pd.merge(features, test_set, how='inner', on='person')

Teniendo el **training_set_global** completo, lo dividimos en dos partes:
1. Un training set propiamente dicho.
2. Un test set.

Esto se hace con la función train_test_split, que recibe:

1. x - Arreglo con todas las columnas del training_set original menos la última (que es el label)

2. y - Arreglo con la última columna del training_set original (que es el label)

3. % del tamaño del test set sobre el total del tamaño del training_set original

De yapa:

z - arreglo con todas las columnas del test_set original. Este data frame tiene las predicciones que tenemos que hacer y comparar en Kaggle para obtener el resultado.

In [33]:
x = np.array(training_set[features_names])
y = np.array(training_set['label'])
z = np.array(test_set[features_names])
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.20)

Defino una función **model_fit()** que recibe el algoritmo clasificador y realiza estas tareas:

1. Fitting: Entrena el algoritmo con el training set X (person + features) e Y (labels)
2. Predicción: Predice los labels en el set de testing
3. Cálculo de accuracy: Calcula el accuracy de la predicción anterior comparando con los labels conocidos (y_test)

La predicción del punto 2 retorna una probabilidad [0, 1] por eso:

4. Clasificación: etiqueta con 0 o 1 dependiendo de la probabilidad.


In [34]:
def modelfit(alg):
    alg.fit(X_train, y_train, eval_metric='aucpr')
    pred = (alg.predict_proba(X_test)[:,1])
    accuracy = accuracy_score(y_test, pred.round(), normalize=False)
    pred_proba = pred
    print("Accuracy: ", accuracy / float(pred.size))

    for i in xrange(0,pred.size):
        if (pred[i] >= 0.5):
            pred[i] = 1
        else: 
            pred[i] = 0

    print('---Classification Report---')
    print(classification_report(y_test,pred))
    print('AUROC :' + str(roc_auc_score(y_test, pred_proba)))